# Twitter Political Affiliation Classification 

## Group members

- Cameron Faulkner
- Nikhil Hegde
- Qianxi Gong
- Atul Nair

### Abstract 
Twitter has become perhaps the preeminent platform for political discourse, where people express their views on a variety of issues. Given the increasing polarization in the political landscape, it is of great interest to understand how people's political views are reflected in their online behavior. Our goal is to develop a model that accurately predicts whether a tweet is posted by a Republican or Democrat, taking into account both the textual content of the tweet and associated numerical features such as the number of retweets, likes, and followers. We seek to identify the best set of features and models that can effectively capture the relationship between the text and numerical features and the political affiliation of tweet authors. To do so, we scraped 786,000 Tweets made by American members of Congress in a two-year span from 2021 to 2023. After vectorizing the text and fine-tuning a Multinomial Naive Bayes algorithm, we were able to accurately classify Tweets as originating from a Republican or Democratic member of Congress at an 82 percent accuracy with a ROC score of 84.  

# Background

Based on existing literature , Several studies have shown promising results using supervised learning models for political affiliation classification based on Twitter data. The results of these studies showed that some models can predict partisan affiliation, with accuracy rates ranging from around 60% to over 90% depending on the specific model and dataset used. For example, in a study by Conover et al. 
(2011), a logistic regression model achieved an accuracy of 86.4% in predicting political orientation (liberal vs. conservative) based on Twitter data [1]. In another study by Barberá et al. (2015), a Random Forest model achieved an accuracy of 87% in predicting US congressional districts' partisan affiliation [2]. They found that incorporating information on retweets and mentions, in addition to tweet text, significantly improved the accuracy of their models [2]. This suggests that the choice of features can play a critical role in the effectiveness of these models.However, an unanswered question in this field is the generalizability of these models to different political contexts or populations. For instance, a study by Mocanu et al. (2015) showed that supervised learning models trained on tweets from the UK Independence Party (UKIP) members may not generalize well to other political groups, such as the Labour Party [3]. Similarly, the effectiveness of these models may vary across different geographical regions, cultures, or languages.

Therefore, while the literature shows promising results for partisan affiliation prediction using supervised machine learning on Twitter, it is important to take into consideration the specific political context and the selection of features to ensure the generalizability and robustness of these models.
We looked at existing research for background information , but our ideas were mostly internally conceptualized. On obtaining the dataset , which had several excellent features , we managed to use our own intuition to build and train the models. The studies cited were mostly background information to give us an idea about the possible pitfalls of our own model


[1] Conover, M. D., Ratkiewicz, J., Francisco, M., Gonçalves, B., Menczer, F., & Flammini, A. (2011). Political polarization on twitter. In Proceedings of the Fifth International AAAI Conference on Weblogs and Social Media (pp. 89-96).
[2] Pak, A., & Paroubek, P. (2010). Twitter as a corpus for sentiment analysis and opinion mining. In LREc (pp. 1320-1326).
[3] Mocanu, D., Baronchelli, A., Perra, N., Gonçalves, B., Zhang, Q., & Vespignani, A. (2015). The Twitter of Babel: Mapping world languages through microblogging platforms. PloS one, 10(11), e0142968. doi: 10.1371/journal.pone.0142968

# Problem Statement

Our goal is to develop a Machine Learning model that predicts whether a tweet is posted by a Republican or Democrat, taking into account both the textual content of the tweet using Semantic Analysis and NLP motivated techniques and associated numerical features such as the number of retweets, likes, and followers. We seek to identify the best set of features and models that can effectively capture the relationship between the text and numerical features and political affiliation of tweet authors which would result in the most accurate predictions using out Evaluation Metrics described below. 

# Data

In order to train our predictive model, we scraped every Tweet made by a member of the American Congress with a Twitter account during the 117th session of Congress, which ran from January 3rd, 2021 to January 3rd, 2023. In total, 530 of 535 members of this session of Congress had Twitter accounts.

We are aware this small collection of users could bias the data. If we were to work with a subset of random Twitter users instead, it would be difficult to manually identify hundreds of Twitter accounts whether they belong to an American political party, or show liberal or conservative viewpoints. Additionally, filtering through bot accounts pose another difficult challenge. 

Gathering data from verified Twitter accounts of members of congress allows us to easily identify political leanings as well and we can extrapolate our findings to the rest of the population as these members of Congress are the public figures of the party. 

Using the dataset of Congressional Twitter handles, we utilized the snscrape Python package to scrape all of the Tweets in our specified time frame from each of the members. This amounted to 786268 Tweets (clearly, members of Congress are prolific Tweeters). [Link to the unprocessed scraped Tweets](https://drive.google.com/file/d/1IUHw2ktOTZc41vi_7Ijrw20-maLJfjTU/view?usp=share_link)

The median amount of Tweets sent by a member of Congress over this span was 1336, corresponding to approximately 1.83 Tweets per day. 


### Removal of Irrelevant Features
For each Tweet in our dataset, we kept the following variables and eliminated some we found irrelevant:

**KEPT VARIABLES**
- `rawContent`: the content of the tweet itself generated by the twitter user 
- `replyCount`: number of replies
- `retweetCount`: number of retweets 
- `likeCount`: number of likes 
- `quoteCount`: number of quotes 
- `links`: any files or resources hyperlinked into the tweet
- `Media`: any videos embedded into the tweet
- `mentionedUsers`: the users mentioned in the tweet 
- `Hashtags`: hashtags appended to the end of the tweet
- `Party`: automated party identification of each member of congress during the data collection process

We elected to drop the following variables as we found them to be redundant or of little utility to our project:

**DROPPED VARIABLES**
- `Url`
- `Date`
- `Id`
- `User`
- `renderedContent`
- `Coordinates`
- `place`
- `Card`
- `viewCount`
- `Vibe`


However, we only used these variables as a starting point for our analysis and needed to treat some of them so that they were usable in our dataset. 

### Preprocessing and Addition of Features

#### TREATMENT OF VARIABLES TAKEN FROM TWITTER SCRAPING

While all of our variables with the word `Count` in them were already in numerical format, we needed to transform the others into numerical data.

For the variables `links`, `Media`, `mentionedUsers`, and `Hashtags`, we weren’t interested in their actual content, but rather how many of them were present in a given tweet (e.g. the number of links placed in a single Tweet). As such, we transformed each variable for every observation into the length of the list of items it pertained to yielding values from 0 and up. 

For `Party` we changed our initial classification of `R` or `D` into 1 and 0, respectively, as we knew that we cannot feed a model categorical data in the form of text. We will be using the party label as the basis of our classifier.

As a fortunate result of using this particular scraper, all of our data was well formatted (no strings were in fields reserved for floats, for example) and, after doing our best practice checks on our data, we found no observations needed to be thrown out.

[link to notebook with implementation of cleaning process described before](https://github.dev/COGS118A/Group040-Wi23/blob/7ee24d6fe6ac05b2b2f63b75e9e21c35270c9291/tweet_cleaning.ipynb)


#### ADDITION OF DERIVED VARIABLES

**Sentiment Analysis**

In addition to the variables provided by our scraping, we conducted sentiment analysis on the text content of each of our Tweets with the theory that Republicans and Democrats may differ in the tone of their communication with the public. 

To apply our sentiment analysis, we took the rawContent variable for each of our Tweets (literally the text of the Tweet) and applied the Vader Sentiment analysis algorithm to them. This algorithm is specifically for the sentiment classification of social media posts, an important factor to consider given how different a Tweet is from a novel. We used this model to classify the sentiment of each Tweet as “positive”, “negative”, or “neutral” based on the scores provided by the algorithm and the thresholds recommended by the developers of the sentiment analysis package. 
	
Again, we ran into the problem of using categorical data in an ML algorithm and so we used one-hot encoding to create three separate binary variables corresponding to each of the three sentiments. 

[link to notebook with sentiment analysis addition](https://github.dev/COGS118A/Group040-Wi23/blob/7ee24d6fe6ac05b2b2f63b75e9e21c35270c9291/sentiment%20analysis%20addition.ipynb)


**Word Frequency and Importance**

We also decided to take further advantage of the rich text content of our dataset and include the frequency and uniqueness of the words that comprised the Tweets in our dataset under the hypothesis that Republicans and Democrats used certain words more frequently than the other party. 

To do so, we used the standard Term Frequency-Inverse Document Frequency measure to quantify the significance of the words used in our Tweets. Understanding that using the algorithm on unprocessed data can yield falsely significant results, we knew we must preprocess the data before applying TF-IDF. We removed stop words, words that commonly occur and offer no value to our classification, using the Natural Language Tool Kit’s (nltk) premade stopwords corpus, and also applied nltk’s Lemmatizer which removes the conjugation of words to avoid duplicates and diluting the significance of them. 

We then took our processed Tweets and vectorized them with sklearn’s TfidfVectorizer and extracted 4500 features from our dataset. The resulting matrix was in the form of columns that contained the TF_IDF scores for each of the 4500 features, for every one of the Tweets. This matrix, added to our 11 other variables, formed our completed dataset.

[link to notebook with text processing and vectorization, and model application](https://github.dev/COGS118A/Group040-Wi23/blob/2a99aec348b73c5eda78ccdd1974db821d0badca/algorithm_application.ipynb)
	
    
**COMPLETED DATASET**

In sum, the dataset we are using for our model has the dimensions of (786268, 4513). This includes the following features: sentiment, engagement metrics from Twitter users, word similarity, and party labels from the Member of Congress who made a given Tweet that will serve as our classification labels.

[Link to completed dataset on Google Drive](https://drive.google.com/file/d/1Z5hXQE7qe--6Cpk_8biCfP_sHRZy7XXb/view?usp=sharing)

Sources/Packages Used:

- [Congressional Twitter Handles](https://frac.org/wp-content/uploads/MOC_Twitter-Handles_117th.pdf)

- [VADER Sentiment Analysis Documentation](https://github.com/cjhutto/vaderSentiment)

- [NLTK Stopwords Documentation](https://www.nltk.org/search.html?q=stopwords&check_keywords=yes&area=default)

- [NLTK Lemmatizer Documentation](https://www.nltk.org/_modules/nltk/stem/wordnet.html)

- [SKLearn TFidfVectorizer Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)


# Proposed Solution

<!-- In this section, clearly describe a solution to the problem. The solution should be applicable to the project domain and appropriate for the dataset(s) or input(s) given. Provide enough detail (e.g., algorithmic description and/or theoretical properties) to convince us that your solution is applicable. Make sure to describe how the solution will be tested.  

If you know details already, describe how (e.g., library used, function calls) you plan to implement the solution in a way that is reproducible.

If it is appropriate to the problem statement, describe a benchmark model<a name="sota"></a>[<sup>[3]</sup>](#sotanote) against which your solution will be compared.  -->


We propose to use a Naive Bayes model to predict the political affiliation (Republican or Democrat) of tweet authors, incorporating both the textual content of the tweet and associated numerical features such as the number of retweets, likes, and replies. The Naive Bayes will allow us to model the fixed effects of the tweet's textual content and numerical features, while accounting for the random effects of individual tweet authors. By including interaction terms between the textual and numerical features, we can capture the non-linear relationships between them and improve the accuracy of our predictions.

To determine the best set of parameters and hyperparameters for the Naive Bayes, we will use k fold cross-validation and grid search techniques. We will divide our dataset into training, validation, and testing sets, and use the training set to fit different Naive Bayes with varying combinations of parameters and hyperparameters. We will then use the validation set to evaluate the performance of each model and select the one with the highest accuracy. Finally, we will test the performance of the selected model on the testing set to ensure its generalizability.

By carefully selecting and tuning our parameters and hyperparameters, we aim to develop an accurate and robust Naive Bayes for predicting the political affiliation of tweet authors based on both textual and numerical features.

# Evaluation Metrics
<!-- !! UPDATE WITH EXAMPLE !!
Propose at least one evaluation metric that can be used to quantify the performance of both the benchmark model and the solution model. The evaluation metric(s) you propose should be appropriate given the context of the data, the problem statement, and the intended solution. Describe how the evaluation metric(s) are derived and provide an example of their mathematical representations (if applicable). Complex evaluation metrics should be clearly defined and quantifiable (can be expressed in mathematical or logical terms). -->


To evaluate the models, we use the Receiver Operating Characteristic (ROC) curve to evaluate the model's performance. 
<!-- 
The Precision-Recall curve is a graphical representation of the precision and recall values of the classifier. Precision is the ratio of correctly predicted positive instances to the total number of predicted positive instances.

$$ Precision = \frac{TP}{TP+FP}  $$ 

Recall is the ratio of correctly predicted positive instances to the total number of actual positive instances. 

$$ Recall = \frac{TP}{TP+FN} $$

The Precision-Recall curve would be useful for evaluating the model's performance when dealing with imbalanced datasets, as it focuses on the precision and recall of the positive class. -->


The ROC curve is a graphical representation of the true positive rate (TPR) against the false positive rate (FPR) for varying classification thresholds. The TPR is the ratio of correctly predicted positive instances to the total number of actual positive instances.

$$ Sensitivity = TPR = \frac{TP}{TP+FN} $$

FPR is the ratio of incorrectly predicted positive instances to the total number of actual negative instances. 

$$ Specificity = 1 - FPR = 1 - \frac{FP}{TN+FP} $$

The area under the ROC curve (ROC AUC) provides an indication of the model's overall performance. A higher ROC AUC score indicates better performance, and a lower score indicates poorer performance.The ROC curve would be useful for evaluating the model's performance when the cost of false positives and false negatives are roughly equal, as it focuses on the TPR and FPR of both classes.

For example, here is the ROC Curve for our benchmark Naive Bayes Algorithm 


![ROC](./ROC.png)


we were able to get an ROC score of 0.75. The closer we get this metric to 1, the better our model will perform. 

<!-- ![cm](./images/cm_example.png)

We can calculate the FPR and TPR as

$$ FPR_{example} = \frac{TP}{TP+FP} = \frac{5100}{5100+1500} = 0.77  $$

$$ TPR_{example} = \frac{TP}{TP+FN} = \frac{5100}{5100+3300} = 0.60  $$ -->





# Results


### Subsection 1: Overview of Approach to Problem

After augmenting our dataset with the process described in depth in our data section, we knew we had to select from algorithms known to perform well on natural language processing tasks. We decided to try three different algorithms: Multinomial Naive Bayes, a random forest ensemble, and stochastic gradient descent (SGD). We recognize that this problem would be a good application of a neural network; however, we felt we lacked the computational power necessary to optimize a neural network as we were already struggling to train our other, less computationally intensive models, on this dataset. 


Having decided on the models we felt were appropriate for this project, we must then determine which model offers the best blend of accuracy and computational efficiency. We want to select the model that offers the best classification performance while not being so computationally expensive that we are unable to evaluate the effect of different hyperparameters to tune it. Finally, using the model that best fits our needs, we will make the final adjustments necessary to optimally answer our question. 


### Subsection 2: Base Model Discussion

Our base model used to compare our other algorithms to was a zero rule algorithm, an algorithm that predicts the classification of a vector by simply guessing the category with the largest number of observations. While a trivial model, it set a benchmark of what one might achieve with an informed guess that we looked to supersede with our more complex models. It also somewhat reflects a key consideration for evaluating our other model, that we must observe not just the overall performance, but also its performance for both class labels.


Upon performing the calculation of the zero rule model, we found it had an overall accuracy of 57.8%. It classified Democrat’s Tweets with 100% accuracy as it was the most represented class, and was 0% accurate in predicting Republicans. We feel confident that we will be able to far exceed this with the implementation of better algorithms.

(base model visible in models.ipynb)


### Subsection 3: Model Selection and Tuning

To select the best model, we decided to train the baseline models of Naive Bayes, Stochastic gradient descent, and random forest classifiers, and then optimize them through Gridsearch and 5-Fold Cross-Validation. We then evaluated the accuracy and ROC score to determine the best model. Each of the models had various parameter grids which heavily affected training time. 


**Naive Bayes**
For Naive Bayes, we tuned the alpha or smoothing parameter with the param grid of {'alpha': [10,1, 0.1, 0.01, 0.001, 0.0001]}  which yielded the best parameter as {'alpha': 0.0001}. The best classification accuracy was 82%. with the ROC Score being 0.84


**SGD**
For SGD, we tuned the loss and penalty with the param grid of params = { 'loss': ['modified_huber', 'log_loss'],  'penalty': ['elasticnet', 'l2']}. The best-performing parameter set was log_loss and ‘elasticnet’. The best classification accuracy was 73%. with the ROC Score being 0.57


**Random Forest**
For Random Forest, I tuned the n_estimators and max_depth with the param grid of params  = { 'n_estimators': [100, 200], 'max_depth': [4, 6] }
 which yielded the best parameter grid as {'max_depth': 6, 'n_estimators': 200} The final classification accuracy was 64%, with the ROC Score being 0.83.

Multinomial Naive Bayes and Random Forest distinguished themselves as being the highest performing of our three models in terms of ROC Score, but we ultimately decided Multinomial Naive Bayes was the best choice of model because of its higher overall accuracy and its substantially lower compute times. 
Reference to code [model.py]


### Subsection 4: Evaluation of Chosen Model

Since the Naive Bayes base model performed better than the Stochastic gradient descent and random forest classifiers on metrics of accuracy and training time, we decided to go forward with tuning the Naive Bayes. We adjusted the learning rate, tried alpha values of 10,1, 0.1, 0.01, 0.001, 0.0001. Found our best alpha value was .0001. Using this learning hyperparameter, we achieved accuracy of 81%. The model correctly identifies 87% of actual class 0 observations and 72% for class 1 observations.



# Discussion

### Interpreting our Result
We determined the best model and parameter combination to solve our problem was Multinomial Naive Bayes with a learning rate of 0.0001. Unfortunately, our best solution was only able to achieve overall accuracy of 82%, and a ROC score of .84. This was not as high as we would have wanted; however, we believe very little additional accuracy may actually be achievable. 


While all members of Congress belong to one of two political parties, their views lie on a continuum. The most conservative Democrat is very ideologically similar, if not more conservative, to the most liberal Republican. For example, the Democratic Senator from West Virginia, Joe Manchin, and the Republican Senator from Maine, Susan Collins both regularly make the headlines for voting against the majority of their respective parties. As such, it may not be reasonable to completely accurately label all members of congress from their Tweets. Unfortunately, it is unclear what percentage of our dataset is not reasonably classifiable.


This similarity between Republicans and Democrats is observable in the word clouds we generated from selecting the top 100 most discriminant word features for each class. We observe that some essential words such as “American” appear as discriminant features for both classes. 
On the bright side, besides these specific words that seem to overlap, the rest of the word cloud looks distinctive for each class. The semantics of words in each word cloud also coincide with common political stereotypes for each party. 


Of note, the model is more capable of identifying democrats than detecting republicans based on the precision and recall score, indicating that there is some room for improvement. This might be due to the imbalance in the dataset stemming from Tweets sent by Democrats being more represented in the dataset solely due to the different Tweeting habits between parties. 


Our final model, thankfully, far exceeds the performance of our base model, indicating that a probability approach like Naive Bayesian is better than Zero Rule Algorithm. This might be due to our customized dataset taking in combinations of features from different dimensions: extracted sentiments, the textual content of the tweet, and associated numerical features. Naive Bayesian approach takes into account the underlying relationship between the features, and this is proven to be more effective in separating the two classes based on evaluating the models' performance. 


### Limitations

While we are glad to have such a large and feature-rich dataset to train our classifiers on, the computational toll it entailed significantly slowed our progress in exploring possible models and then tuning them. We quickly realized we lacked the computational power necessary to implement a neural network, a model which we felt was a good fit for this dataset, and generally struggled to efficiently test various hyperparameters for models with lower resource demands. With greater computational resources, we feel we could have potentially been able to select and develop a more accurate classifier that could’ve potentially solved some of the issues we experienced with differing predictive performance between labels, although we believe our solution still performed very well on a highly difficult classification task. 


### Ethics & Privacy

If someone were to use such an algorithm to identify the partisan affiliation of someone they know based on their Twitter activity, it could lead to that individual being singled out and potentially discriminated against. For example, they might face social ostracism or even job discrimination based on their political views, which is a clear violation of their rights to free speech and privacy. This is precisely why we have secret ballots in the US. The secret ballot ensures that citizens can express their political views without fear of retaliation or discrimination. It allows individuals to vote for the candidate or party that they truly support without the risk of being singled out or punished for their political views. In the case of using Twitter data to predict political affiliations, it is important to consider the potential implications of such models on individuals' privacy and rights. While it can be tempting to use this kind of data to gain insights into people's political views, it is important to approach it with caution and respect for individuals' privacy.

**Data Collection and Usage:** One important ethical consideration is how the data is collected and used. For example, is the data being collected with the user's consent? Is it being used for purposes other than what was initially intended? It's important to ensure that data is collected and used in an ethical and transparent manner. While these tweets are obtained legally from politicians who know what they're doing when they post online , there's an implication that the average joe could have their data being used against them. 

**Bias in Predictive Models:** Another ethical concern is the potential for bias in predictive models. These models may be trained on biased data, which could result in inaccurate predictions and reinforce existing biases. It's important to test and validate predictive models to ensure that they are fair and accurate.

**Role of Social Media Companies:** Social media companies play a key role in collecting and sharing data. It's important for these companies to have policies and practices in place to protect users' data and privacy. This includes being transparent about how data is collected and used, and providing users with options to control their data.

**Our dataset is publicly avaiable.** So are a huge chunk of tweets thanks to twitter's API. There's not much we can do to prevent the data from falling into the wrong hands, the bigger concern is the model being misuses. 

#### How are we going to handle ethics ? 

**Model training and evaluation**: We will use techniques such as cross-validation to evaluate the performance of the model and identify any biases that may exist. 

**Deployment and monitoring**: We will monitor the performance of the model in production and identify any unintended consequences or ethical issues that may arise. We will also implement mechanisms to prevent the misuse of the model, such as restricting access to the model or using it only for specific purposes.

**Regular review**: We will regularly review the model's performance and re-evaluate its ethical implications, as well as update the model to address any new concerns that may arise.

### Conclusion

We found that we were able to correctly classify the party of a Member of Congress making a Tweet 82% of the time albeit with some deviation in the correct identification of Republicans and Democrats. Given the overlapping political ideologies of different members of Congress and the potential for them to occasionally Tweet messages that have little relevance to their political views, we believe this to be a rather successful classifier. However, we still believe there is potential for a better solution to this problem. Although there is no guarantee of better performance, if we had access to more processing power, we would have implemented an evaluated a neural network. As it stands, however, we developed a promising classifier that we believe is not likely to be drastically superseded by other implemenations. 

# Footnotes
<a name="lorenznote"></a>1.[^] Antypas D. , Preece A., Camacho-Collados J. “Politics, Sentiment and Virality: A Large-Scale Multilingual Twitter Analysis in Greece, Spain and United Kingdom”. Online Social Networks and Media. August 22, 2022. https://arxiv.org/pdf/2202.00396.pdf

<a name="admonishnote"></a>2.[^] Gangwar, A. and Mehta,T.. “Sentiment Analysis of Political Tweets for Israel using Machine Learning”. LearnByResearch. April, 2022. https://arxiv.org/pdf/2204.06515.pdf

